In [1]:
# 1. Instalação (Rode primeiro)
!pip install -q transformers accelerate bitsandbytes

import torch
import os
import gc
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Limpeza preventiva de memória Python
gc.collect()
torch.cuda.empty_cache()

# ==========================================
# CONFIGURAÇÃO DE MODELO (QWEN 1.5B - ULTRA LEVE)
# ==========================================
model_id = "Qwen/Qwen2.5-1.5B-Instruct"

print(f"Carregando {model_id}...")
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    load_in_4bit=True,
    torch_dtype=torch.float16,
    trust_remote_code=True
)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# ==========================================
# LEITURA DOS ARQUIVOS
# ==========================================
def ler_arquivo(nome):
    if os.path.exists(nome):
        with open(nome, "r", encoding="utf-8") as f:
            return f.read()
    return ""

# Lendo e limitando o tamanho para não estourar o contexto
c_releases = ler_arquivo("releases.txt")[:4000]

prompt = f"""Você é um analista de software sênior. Analise os logs do projeto Screenpipe:

DADOS DE RELEASES:
{c_releases}

TAREFA:
1. Descreva detalhadamente a estratégia de versionamento e cadência de releases.
2. Identifique o modelo de workflow (ex: Trunk-based, GitFlow) e como os desenvolvedores colaboram.
Dê uma resposta técnica estruturada."""

messages = [{"role": "user", "content": prompt}]
prompt_final = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# ==========================================
# EXECUÇÃO (COM GERENCIAMENTO DE MEMÓRIA)
# ==========================================
print("\n--- INICIANDO ANÁLISE INTEGRADA ---\n")

with torch.no_grad():
    saida = pipe(
        prompt_final,
        max_new_tokens=1000,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        return_full_text=False
    )

print("\n=== RESULTADO DA ANÁLISE ===\n")
print(saida[0]['generated_text'])

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.9 MB/s eta 0:00:00
Carregando Qwen/Qwen2.5-1.5B-Instruct...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Device set to use cuda:0



--- INICIANDO ANÁLISE INTEGRADA ---


=== RESULTADO DA ANÁLISE ===

### Estratégia de Versionamento e Cadência de Releases

A estratégia de versionamento e cadência de releases em projetos complexos como o ScreenPipe envolve várias etapas técnicas e estratégicas. Aqui está um esboço da estratégia para facilitar a compreensão e implementação.

#### 1. Estrategias de Versionamento e Cadência de Releases

**a)** **Versionamento**: O ScreenPipe utiliza uma estrutura de versões manuais, onde cada release tem um número específico. Este método permite que cada release tenha um identificador único, facilitando a identificação e gerenciamento dos releases.

**b)** **Cadência de Releases**: A cadência de releases depende do progresso do projeto e das necessidades específicas do cliente ou equipe. Em geral, releases são criados quando houver novas funcionalidades ou correções de bugs.

#### 2. Modelos de Workflow e Colaboração

**Modelo GitFlow**:
- **Trunk-Based**: O projeto mantém um branch pr